# Hyperparameter optimization with Optuna

## Data

In [1]:
from reservoirpy.datasets import lorenz, rossler
from rcpy.data.data_generation import ulam_map
from rcpy.data import preprocess_data_rcpy 


# ----- Generate data -----

#data_raw = ulam_map(x0=0.1, n_steps=10000, discard=1000)
#data_raw = lorenz(10000)
data_raw = rossler(11000)[1000:,0]


# ----- Preprocess data -----

# Parameters for preprocessing
offset = 0
train_length = 6000
val_length = 1000

# Preprocess the raw data
data = preprocess_data_rcpy(
    data=data_raw,
    init_discard=offset,
    train_length=train_length,
    val_length=val_length,
    normalize=True
)

## Hypopt

In [2]:
from rcpy.hypopt import build_objective, run_optimization

train_config = {
    "washout_training": 500
}

forecast_config = {
    "forecasting_step": 1,
    "warmup_length": 1000
}

trials = 50
timeout_hours = 2

loss_config = {
    "function": "soft_horizon",
    "threshold": 0.2,
    "softness": 0.02
}

hypopt_search_space = {
    "spectral_radius": {"range": [0.1, 2], "log": True},
    "leak_rate": {"fixed": 1},
    "input_scaling": {"range": [0.01, 1]},
    "alpha": {"range": [1e-8, 1e-2]},
    "p": {"fixed": 0.1}
}


hypopt_config = {
    "trials": trials,
    "timeout_hours": timeout_hours,
    "loss": loss_config,
    "search_space": hypopt_search_space
}

seed = 9610
reservoir_units = 1000
study_name = "esn_hypopt_study"
#db_file = "esn_hypopt_study.db"
db_file = None

objective_func = build_objective(
    data=data,
    reservoir_units=reservoir_units,
    seed=seed,
    hypopt_config=hypopt_config,
    train_config=train_config,
    forecast_config=forecast_config,
)

study = run_optimization(
    study_name=study_name,
    db_file=db_file,   # or db_file if you want persistence
    objective_func=objective_func,
    hypopt_config=hypopt_config
)

/home/user/miniforge3/envs/esn-enso/lib/python3.10/site-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-28 17:06:03,013] A new study created in memory with name: esn_hypopt_study
[I 2025-11-28 17:06:04,916] Trial 0 finished with value: -11.455526504815252 and parameters: {'spectral_radius': 0.21757250105504664, 'input_scaling': 0.7950465423887378, 'alpha': 0.008213817546595479}. Best is trial 0 with value: -11.455526504815252.
[I 2025-11-28 17:06:06,773] Trial 1 finished with value: -24.447814718119396 and parameters: {'spectral_radius': 0.8529787921560691, 'input_scaling': 0.12799540116532773, 'alpha': 0.006295771741512489}. Best is trial 1 with value: -24.447814718119396.
[I 2025-11-28 17:06:08,714] Trial 2 finished with value: -20.368754456666906 and parameters: {'spectral_radius': 0.25746453360416766, 'input_scaling': 0.9650263712446414, 'alpha': 0.004

## Best hyperparameters

In [8]:
import optuna
import matplotlib.pyplot as plt

best_params = study.best_params
print("Best hyperparameters found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

fig = optuna.visualization.plot_optimization_history(study)
#fig.update_yaxes(type='log')
fig.show()

Best hyperparameters found:
  spectral_radius: 1.2241171268225497
  input_scaling: 0.9551154044596366
  alpha: 0.0071692167610677504
